In [70]:
import nltk
import string
import jsonrpclib
from simplejson import loads
from nltk.corpus import wordnet as wn
import re
from string import punctuation
server = jsonrpclib.Server("http://localhost:8080")

r = re.compile(r'[{}]'.format(punctuation))

name appears in the paragraph before


2 alternant lines 

if the name appears in the previous or following utterance with a pattern such as ',name,'

the object of the speech verb


In [124]:
f = open('annotation_speaker')
n = 0
instance = []
temp = []
speaker = []
p = 0
keyposition = []   #the linenum of the paragraph to be treated
for line in f:
    if n <= 31080:
        if '###############' in line:
            keyposition.append(p)
            current = [temp.pop()]
            previous = temp[:]
            temp = []
            continue
        if 'speaker:	' in line:
            speaker.append(line.split('speaker:	')[1].split('\r\n')[0])
            instance.append([current,previous,temp])
            temp = []
            p = 0
            continue
        if line.replace(' ','') != '\r\n':
            temp.append(line.replace('\r\n',''))
            p = p+1
    n = n+1
print instance[10]


#instance [0,1,2]:
#0 : current paragraph (including the object utterance)
#1 : 9 previous paragraphs
#2 : 5 following paragraphs


[['    "I want these letters burned, every one," Cersei declared. "No hint of this must reach my son\'s ears, or my father\'s."'], ["    He knew that shadow. As he knew the man who'd cast it.", '    TYRION', '    The queen was not disposed to wait on Varys. "Treason is vile enough," she declared furiously, "but this is barefaced naked villainy, and I do not need that mincing eunuch to tell me what must be done with villains."', "    Tyrion took the letters from his sister's hand and compared them side by side. There were two copies, the words exactly alike, though they had been written by different hands.", '    "Maester Frenken received the first missive at Castle Stokeworth," Grand Maester Pycelle explained. "The second copy came through Lord Gyles."', '    Littlefinger fingered his beard. "If Stannis bothered with them, it\'s past certain every other lord in the Seven Kingdoms saw a copy as well."'], ['    "I imagine Father\'s heard rather more than a hint by now," Tyrion said dryly

In [60]:
result = loads(server.parse(instance[0][1][7]))
print result['sentences'][0]['parsetree'].split('] [')

[u'[Text=Cortnay CharacterOffsetBegin=8 CharacterOffsetEnd=15 PartOfSpeech=NNP Lemma=Cortnay NamedEntityTag=PERSON', u'Text=Penrose CharacterOffsetBegin=16 CharacterOffsetEnd=23 PartOfSpeech=NNP Lemma=Penrose NamedEntityTag=PERSON', u'Text=ignored CharacterOffsetBegin=24 CharacterOffsetEnd=31 PartOfSpeech=VBD Lemma=ignore NamedEntityTag=O', u'Text=him CharacterOffsetBegin=32 CharacterOffsetEnd=35 PartOfSpeech=PRP Lemma=he NamedEntityTag=O', u'Text=, CharacterOffsetBegin=35 CharacterOffsetEnd=36 PartOfSpeech=, Lemma=, NamedEntityTag=O', u'Text=preferring CharacterOffsetBegin=37 CharacterOffsetEnd=47 PartOfSpeech=VBG Lemma=prefer NamedEntityTag=O', u'Text=to CharacterOffsetBegin=48 CharacterOffsetEnd=50 PartOfSpeech=TO Lemma=to NamedEntityTag=O', u'Text=address CharacterOffsetBegin=51 CharacterOffsetEnd=58 PartOfSpeech=VB Lemma=address NamedEntityTag=O', u'Text=Stannis CharacterOffsetBegin=59 CharacterOffsetEnd=66 PartOfSpeech=NNP Lemma=Stannis NamedEntityTag=O', u'Text=. CharacterOffset

In [109]:
def countwordperline(instance,num_instance):
    numlist = []
    for i in range(3):
        for j in range(len(instance[i])):
            new_strs = r.sub(' ',instance[i][j])
            num = len(new_strs.split())
            numlist.append(num)
    numlist.insert(keyposition[num_instance]-1, numlist.pop(0))
    return numlist
            
countwordperline(instance[0],0)

[62, 102, 138, 100, 44, 9, 102, 78, 33, 21, 13, 17, 61, 6, 64]

In [168]:
def convertoffset(numlist,linenum,offset,sentence):
    temp = sentence[0:offset]
    new_strs = r.sub(' ',temp)
    num = len(new_strs.split())
    pos = 0
    for i in range(linenum-1):
        pos = pos + numlist[i]
    return pos+num+1

convertoffset(countwordperline(instance[0],0),keyposition[0],33,instance[0][0][0])

676

In [179]:
def ner(instance):
    entities = []
    n = 0
    for obj in instance:
        numlist = countwordperline(obj,n)
        print 'progress : '+str(float(n)/float(len(instance))*100)+'%\n'
        current = [[]]
        temp = []
        for z in range(len(loads(server.parse(obj[0][0]))['sentences'])):
            temp = temp + loads(server.parse(obj[0][0]))['sentences'][z]['parsetree'].split('] [')
        for frag in temp:
            if 'PartOfSpeech=NNP' in frag:
                text = frag.split('Text=')[1].split(' ')[0]
                position = frag.split('CharacterOffsetBegin=')[1].split(' ')[0]
                pos = convertoffset(numlist,keyposition[n],int(position),obj[0][0])
                current[0].append([text,int(position),pos])
            elif 'PartOfSpeech=N' in frag:
                text = frag.split('Text=')[1].split(' ')[0]
                position = frag.split('CharacterOffsetBegin=')[1].split(' ')[0]
                pos = convertoffset(numlist,keyposition[n],int(position),obj[0][0])
                lex = []
                for synset in wn.synsets(text):
                    lex.append(synset.lexname())
                if lex == []:
                    continue
                elif float(lex.count('noun.person'))/float(len(lex)) >= 0.4:
                    current[0].append([text,int(position),pos])
        previous = []
        for i in range(len(obj[1])):
            previous.append([])
            temp = []
            for x in range(len(loads(server.parse(obj[1][i]))['sentences'])):
                try:
                    temp = temp + loads(server.parse(obj[1][i]))['sentences'][x]['parsetree'].split('] [')
                except IndexError:
                    continue
            for frag in temp:
                if 'PartOfSpeech=NNP' in frag:
                    text = frag.split('Text=')[1].split(' ')[0]
                    position = frag.split('CharacterOffsetBegin=')[1].split(' ')[0]
                    pos = convertoffset(numlist,i+1,int(position),obj[1][i])
                    previous[i].append([text,int(position),pos])
                elif 'PartOfSpeech=N' in frag:
                    text = frag.split('Text=')[1].split(' ')[0]
                    position = frag.split('CharacterOffsetBegin=')[1].split(' ')[0]
                    pos = convertoffset(numlist,i+1,int(position),obj[1][i])
                    lex = []
                    for synset in wn.synsets(text):
                        lex.append(synset.lexname())
                    if lex == []:
                        continue
                    elif float(lex.count('noun.person'))/float(len(lex)) >= 0.4:
                        previous[i].append([text,int(position),pos])
        following = []
        for j in range(len(obj[2])):
            following.append([])
            temp = []
            for y in range(len(loads(server.parse(obj[2][j]))['sentences'])):
                try:
                    temp = temp + loads(server.parse(obj[2][j]))['sentences'][y]['parsetree'].split('] [')
                except IndexError:
                    continue
            for frag in temp:
                if 'PartOfSpeech=NNP' in frag:
                    text = frag.split('Text=')[1].split(' ')[0]
                    position = frag.split('CharacterOffsetBegin=')[1].split(' ')[0]
                    pos = convertoffset(numlist,keyposition[n]+j+1,int(position),obj[2][j])
                    following[j].append([text,int(position),pos])
                elif 'PartOfSpeech=N' in frag:
                    text = frag.split('Text=')[1].split(' ')[0]
                    position = frag.split('CharacterOffsetBegin=')[1].split(' ')[0]
                    pos = convertoffset(numlist,keyposition[n]+j+1,int(position),obj[2][j])
                    lex = []
                    for synset in wn.synsets(text):
                        lex.append(synset.lexname())
                    if lex == []:
                        continue
                    if float(lex.count('noun.person'))/float(len(lex)) >= 0.4:
                        following[j].append([text,int(position),pos])
        entities.append([current,previous,following])
        n = n +1
        return entities

entities = ner(instance)
entities_new = entities[:]
print entities

#entities [name, offset in the paragraph, word position in the instance]

progress : 0.0%

[[[[[u'lady', 33, 676], [u'Ser', 40, 677], [u'Cortnay', 44, 678], [u'gods', 82, 685], [u'king', 104, 689]]], [[[u'Davos', 74, 14], [u'king', 306, 58]], [[u'man', 10, 64], [u'Davos', 71, 76], [u'Seaworth', 77, 77], [u'lords', 101, 81], [u'beasts', 292, 111], [u'Davos', 392, 128], [u'king', 403, 130]], [[u'Davos', 25, 169], [u'king', 219, 210], [u'king', 242, 214], [u'Davos', 267, 218], [u'son', 290, 222], [u'Devan', 294, 223], [u'squires', 318, 228], [u'Stannis', 336, 231], [u'Baratheon', 344, 232], [u'Dragonstone', 495, 260], [u'king', 523, 264], [u'Lord', 559, 270], [u'Renly', 564, 271], [u'boy', 626, 281], [u'father', 650, 286], [u'Maester', 659, 287], [u'Lady', 705, 296], [u'Melisandre', 710, 297]], [[u'son', 203, 343], [u'boy', 225, 348], [u'father', 292, 361], [u'Lord', 359, 373], [u'king', 402, 383], [u'squire', 409, 385], [u'king', 483, 400], [u'god', 490, 402]], [[u'Stannis', 97, 421], [u'Ser', 142, 429], [u'Cortnay', 146, 430], [u'standard-bearer', 162, 433]],

In [180]:
#combine contiguous name entities and separate those within and outside quotation marks
def contiguous(instance,namelist,ni,cls,nl):
    i = 0
    t = len(namelist)-1
    newlist = {}
    newlist['in'] = []
    newlist['out'] = []
    while i < t:
        if namelist[i+1][1]-namelist[i][1] == len(namelist[i][0])+1 and instance[ni][cls][nl][namelist[i][1]+len(namelist[i][0])] == ' ':
            namelist[i][0] = namelist[i][0]+' '+namelist[i+1][0]
            namelist.pop(i+1)
            t = t - 1
            i = i - 1
        i = i + 1
    for obj in namelist:
        if instance[ni][cls][nl][0:obj[1]].count('"')%2 == 1:
            newlist['in'].append(obj)
        else:
            newlist['out'].append(obj)
    return newlist

for j in range(3):
    for k in range(len(instance[0][j])):
        entities_new[0][j][k] = contiguous(instance,entities[0][j][k],0,j,k)

In [181]:
print entities_new[0][0][0]
print instance[0][0][0]

{'out': [[u'Ser Cortnay', 40, 677]], 'in': [[u'lady', 33, 676], [u'gods', 82, 685], [u'king', 104, 689]]}
    "I wish you well of them, my lady," Ser Cortnay answered, "but I bow to other gods, and a different king."


In [182]:
print entities_new

[[[{'out': [[u'Ser Cortnay', 40, 677]], 'in': [[u'lady', 33, 676], [u'gods', 82, 685], [u'king', 104, 689]]}], [{'out': [[u'Davos', 74, 14], [u'king', 306, 58]], 'in': []}, {'out': [[u'man', 10, 64], [u'Davos Seaworth', 71, 76], [u'lords', 101, 81], [u'beasts', 292, 111], [u'Davos', 392, 128], [u'king', 403, 130]], 'in': []}, {'out': [[u'Davos', 25, 169], [u'king', 219, 210], [u'king', 242, 214], [u'Davos', 267, 218], [u'son Devan', 290, 222], [u'squires', 318, 228], [u'Stannis Baratheon', 336, 231], [u'Dragonstone', 495, 260], [u'king', 523, 264], [u'boy', 626, 281], [u'father', 650, 286]], 'in': [[u'Lord Renly', 559, 270], [u'Maester', 659, 287], [u'Lady Melisandre', 705, 296]]}, {'out': [[u'son', 203, 343], [u'boy', 225, 348], [u'father', 292, 361], [u'Lord', 359, 373], [u'king', 402, 383], [u'squire', 409, 385], [u'king', 483, 400], [u'god', 490, 402]], 'in': []}, {'out': [[u'Stannis', 97, 421], [u'Ser Cortnay', 142, 429], [u'standard-bearer', 162, 433]], 'in': []}, {'out': [], 'in

In [183]:
entities_out = []
for i in range(len(entities_new)):
    entities_out.append([])
    for j in range(3):
        entities_out[i].append([])
        for k in range(len(entities_new[i][j])):
            entities_out[i][j].append([])
            entities_out[i][j][k] = entities_new[i][j][k]['out']
print entities_out

[[[[[u'Ser Cortnay', 40, 677]]], [[[u'Davos', 74, 14], [u'king', 306, 58]], [[u'man', 10, 64], [u'Davos Seaworth', 71, 76], [u'lords', 101, 81], [u'beasts', 292, 111], [u'Davos', 392, 128], [u'king', 403, 130]], [[u'Davos', 25, 169], [u'king', 219, 210], [u'king', 242, 214], [u'Davos', 267, 218], [u'son Devan', 290, 222], [u'squires', 318, 228], [u'Stannis Baratheon', 336, 231], [u'Dragonstone', 495, 260], [u'king', 523, 264], [u'boy', 626, 281], [u'father', 650, 286]], [[u'son', 203, 343], [u'boy', 225, 348], [u'father', 292, 361], [u'Lord', 359, 373], [u'king', 402, 383], [u'squire', 409, 385], [u'king', 483, 400], [u'god', 490, 402]], [[u'Stannis', 97, 421], [u'Ser Cortnay', 142, 429], [u'standard-bearer', 162, 433]], [], [[u'Lord Florent', 70, 468], [u'fox', 95, 473], [u'Lord', 234, 497], [u'Brightwater', 242, 499], [u'Renly', 281, 506], [u'Stannis', 314, 512], [u'gods', 357, 520], [u'Lord', 378, 525], [u'queen', 414, 532], [u'Dragonstone', 423, 534], [u'uncle Axell', 450, 538], [u

In [195]:
def distance(entities,num_instance):
    sentence = instance[num_instance][0][0]
    if sentence.count('"') == 1:
        offset = sentence.index('"')
        position = [convertoffset(countwordperline(instance[num_instance],num_instance),keyposition[num_instance],offset,instance[num_instance][0][0])]
    else:
        offset = []
        p = 0
        for c in sentence:
            if c == '"':
                offset.append(p)
            p = p+1
        position = []
        for i in range(2):
            position.append(convertoffset(countwordperline(instance[num_instance],num_instance),keyposition[num_instance],offset[i],instance[num_instance][0][0]))
    dist = []
    for i in range(3):
        for j in range(len(entities[i])):
            for k in range(len(entities[i][j])):
                pos = entities[i][j][k][2]
                if len(position) == 1:
                    dist.append([abs(pos-position[0])])
                else:
                    if pos <= position[0]:
                        dist.append([position[0]-pos])
                    elif pos >= position[1]:
                        dist.append([pos-position[1]])
    return dist
                
feature = distance(entities_out[0],0)
order = feature[:]
for i in range(len(feature)):
    order[i] = [order[i][0],i]
order.sort()
print order

[[0, 0], [24, 47], [52, 48], [103, 46], [108, 49], [110, 45], [114, 50], [116, 44], [125, 43], [127, 42], [131, 41], [135, 40], [137, 39], [144, 38], [149, 37], [157, 36], [163, 35], [170, 34], [172, 33], [196, 32], [201, 31], [236, 30], [240, 29], [248, 28], [267, 27], [269, 26], [284, 25], [286, 24], [296, 23], [308, 22], [321, 21], [326, 20], [383, 19], [388, 18], [405, 17], [409, 16], [438, 15], [441, 14], [447, 13], [451, 12], [455, 11], [459, 10], [500, 9], [539, 8], [541, 7], [558, 6], [588, 5], [593, 4], [605, 3], [611, 2], [655, 1]]


In [196]:
for i in range(len(order)):
    feature[order[i][1]].append(i+1)
print feature

[[0, 1], [655, 51], [611, 50], [605, 49], [593, 48], [588, 47], [558, 46], [541, 45], [539, 44], [500, 43], [459, 42], [455, 41], [451, 40], [447, 39], [441, 38], [438, 37], [409, 36], [405, 35], [388, 34], [383, 33], [326, 32], [321, 31], [308, 30], [296, 29], [286, 28], [284, 27], [269, 26], [267, 25], [248, 24], [240, 23], [236, 22], [201, 21], [196, 20], [172, 19], [170, 18], [163, 17], [157, 16], [149, 15], [144, 14], [137, 13], [135, 12], [131, 11], [127, 10], [125, 9], [116, 8], [110, 6], [103, 4], [24, 2], [52, 3], [108, 5], [114, 7]]


In [188]:
#feature [distance from the quote, the ordinal position]